In [ ]:
import datetime
import pandas as pd
import math
import sys

In [ ]:
file_name = '2022_01_13_Spot1.xlsx'
option_file_name  = '2022_01_13_Options.xlsx'
rate = 0.2
 


In [ ]:
def trading_cost(type,A):
    Quantity= 25
    Brokerage = 20/Quantity
    Exchange_comission = 0.00053*A
    SEBI_Charges = 0.000001*A
    GST = 0.18*(Brokerage+Exchange_comission+SEBI_Charges)
    StampDuty=0
    STT = 0
    if(type=="Buy"):
       StampDuty = 0.00003*A
    if(type=="Sell"):
       STT = 0.0005*A
       
    cost = Brokerage + Exchange_comission + StampDuty + SEBI_Charges + GST + STT

    return round(cost,2)
     


In [ ]:
def final_stop(file):
    a = file.split('_')
    final = "20"+a[0]+"-"+a[1]+"-"+a[2]+" 15:25:00"
    return final

def CMPFromExcel(i,df):

    return float(df.iloc[i+1,10])

def TimeFromExcel(i,df):

    return df.iloc[i+1,0]

def CallPremFromExcel(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellC(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
    
def CallPremFromExcel2(i,df):


    return float(df.iloc[i,10])

def CallPremFromExcel2High(i,df):

    return float(df.iloc[i,8])
            
def PutPremFromExcel(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and  df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellP(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
def PutPremFromExcel2High(i,df):

    return float(df.iloc[i,8])
            
def PutPremFromExcel2(i,df):
    
    return float(df.iloc[i,10])

def findJ(startTime,df):
    final_stopTime = final_stop(file_name)
    if(str(startTime).split()==str(final_stopTime).split()):
        sys.exit()
    for j in range(df.iloc[-1,13]):
        if(df.iloc[j,0]==startTime  ):
            return j-1

def today(date):
    date =str(date).split('-')
    month = int(date[1])
    day = int(date[2])
    year = int(date[0])
    a = datetime.date(year,month,day).weekday()
    if a == 0:
        x  = "Monday"

    if a == 1:
        x  ="Tuesday" 

    if a==2:
        x  ="Wednesday" 

    if a==3:
       x ="Thursday" 

    if a==4:
       x ="Friday"

    if a==5:
       x = "Saturday" 

    if a==6: 
        x  = "Sunday"
    return x
    


In [ ]:

final_stopTime = final_stop(file_name)

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)

lotsize = 25
no_of_lot = 1
qty = no_of_lot*lotsize


n=1
j = 100
t = TimeFromExcel(j,dfs)
day = str(t).split()

stopTime = day[0]+" 15:25:00"
startTime = day[0]+" 09:20:00"

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
print("dataframes ready")

j = findJ(startTime,dfs)

cmp = CMPFromExcel(j,dfs)

t = TimeFromExcel(j,dfs)
print(t)
bufferZone = "|"+"".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+"".center(21)+"|\n"
dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
dfc = pd.read_excel(option_file_name,sheet_name='Call',engine='openpyxl',dtype=object,header=None)
dfp = pd.read_excel(option_file_name,sheet_name='Put',engine='openpyxl',dtype=object,header=None)

def start(j,prev_table,total,count,dfc,dfp,dfs):
    count+=1

    initial = j

    cmp = CMPFromExcel(j,dfs)
     
    t = TimeFromExcel(j,dfs)
    day = str(t).split()
    myWeekday = today(day[0])

    stopTime = day[0]+" 15:25:00"
    startTime = day[0]+" 09:20:00"

    zintermediate = (math.floor(cmp/100))*100
    x = cmp%100
    y = math.floor(x)

    if (y>=50):
        strikeprice = zintermediate+100
    else:
        strikeprice = zintermediate
    SP = str(strikeprice)

    iP = getFirstCellP(SP,t,dfp)
    iC = getFirstCellC(SP,t,dfc)

    PE = "CNXBAN "+str(strikeprice)+" PE"
    CE = "CNXBAN "+str(strikeprice)+" CE"



    execPremCall = CallPremFromExcel(SP,t,dfc)
    execPremCall = float(execPremCall)
    execPremPut = PutPremFromExcel(SP,t,dfp)
    execPremPut = float(execPremPut)

    Entry_CostCall  = trading_cost("Sell",execPremCall)
    Entry_CostPut = trading_cost("Sell",execPremPut)





    # Unrealised Profit / (Loss) = Executed Premium - Current Premium
    # Realised Profit / (Loss) = Executed Premium - Stop Loss Premium

    #                                     Premium realtime

    #                                       stop loss

    
    
    stlp_1C = execPremCall*(1+rate)
    stlp_1P = execPremPut*(1+rate)
    
    #stop loss trial
    putStoplossIsHit = False
    CallStoplossIsHit = False

    exitPremPut = 0
    exitPremCall = 0

    # str(qty).center(12)+"|"+


    
    
    while(True):

        
        if(PutPremFromExcel2High(iP,dfp)>=stlp_1P ):
            putStoplossIsHit = True
            
            exitTimePut = TimeFromExcel(j,dfs)
            exitPremPut = min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
            
            exitCostPut = trading_cost("Buy",exitPremPut)
            totalCostPut = Entry_CostPut +exitCostPut
            Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut


            break

        if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):
       
            exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
            exitTimeCall = TimeFromExcel(j,dfs)

            exitCostCall = trading_cost("Buy",exitPremCall)
            totalCostCall = exitCostCall+Entry_CostCall
            Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall

            break
        if(TimeFromExcel(j,dfs) == stopTime):

            putStoplossIsHit = True
            exitPremCall = CallPremFromExcel2(iC,dfc)
            exitPremPut = PutPremFromExcel2(iP,dfp)
            exitTimeCall =exitTimePut= TimeFromExcel(j,dfs)
            

            exitCostCall = trading_cost("Buy",exitPremCall)
            exitCostPut = trading_cost("Buy",exitPremPut)

            totalCostCall = Entry_CostCall+exitCostCall
            totalCostPut = Entry_CostPut +exitCostPut

            Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
            Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
            print("FINAL TABLE".center(200,'*'))
            print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

            print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


            print(prev_table)
            print("|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(PutPremFromExcel2(iP,dfp)).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
            print("|"+CE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
            print(bufferZone)
            print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")
        

            print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
            print("Day Finished")

            if(count<5 and j+37<dfs.iloc[-1,13] ):
                if(stopTime == final_stopTime):
                    print("ALL DAYS FINISHED")
                    sys.exit()
                t = TimeFromExcel(j+37,dfs)
                day = str(t).split()

                startTime = day[0]+" 09:20:00"

                j = findJ(startTime,dfs)
                start(j,"",0,count,dfc,dfp,dfs)

            else:
                print("ALL DAYS FINISHED")
                sys.exit()




        Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)
        Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)


        iP =iP+n
        iC = iC+n
        j=j+n       # time.sleep(trialTime*60)


    if(putStoplossIsHit):





        while(True):
            


            
            if(CMPFromExcel(j,dfs)>=cmp):    # or prem1*(1.1)>=prem
                cmp = CMPFromExcel(j,dfs)
            
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)


                
                if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):

                    exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    exitCostCall = trading_cost("Buy",exitPremCall)
                    totalCostCall = exitCostCall+Entry_CostCall
                    
                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall


                    break
                if(TimeFromExcel(j,dfs) == stopTime):

                        exitPremCall = CallPremFromExcel2(iC,dfc)
                        exitTimeCall = TimeFromExcel(j,dfs)

                        exitCostCall = trading_cost("Buy",exitPremCall)
                        totalCostCall = exitCostCall+Entry_CostCall

                        Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                        Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
                        print("FINAL TABLE".center(200,'*'))
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


                        print(prev_table)
                        print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                        print(bufferZone)
                        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                        print("Day Finished")
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print("ALL DAYS FINISHED")
                                sys.exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            startTime = day[0]+" 09:20:00"

                            j = findJ(startTime,dfs)


                            start(j,"",0,count,dfc,dfp,dfs)
                        else:
                            print("ALL DAYS FINISHED")
                            sys.exit()


                    
            



            else:
                if(stlp_1C>CallPremFromExcel2(iC,dfc)*(rate+1)):
                    stlp_1C=CallPremFromExcel2(iC,dfc)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremCall = CallPremFromExcel2(iC,dfc)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    exitCostCall = trading_cost("Buy",exitPremCall)
                    totalCostCall = exitCostCall+Entry_CostCall

                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                    print("FINAL TABLE".center(200,'*'))
                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                    print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


                    print(prev_table)
                    print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                    print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                    print(bufferZone)
                    print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    print("Day Finished")
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print("ALL DAYS FINISHED")
                            sys.exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+" 09:20:00"

                        j = findJ(startTime,dfs)

                        
                        start(j,"",0,count,dfc,dfp,dfs)
                    else:
                        print("ALL DAYS FINISHED")
                        sys.exit()

                cmp = CMPFromExcel(j,dfs)
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)


            j=j+n
            iP =iP+n
            iC = iC+n
            # time.sleep(trialTime*60)
            
    else:
   
        while(True):
            



            
            if(CMPFromExcel(j,dfs)<=cmp ): 
                cmp = CMPFromExcel(j,dfs)



                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)




                if(PutPremFromExcel2High(iP,dfp)>=stlp_1P):
                    putStoplossIsHit = True
                    exitPremPut =min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
                    exitTimePut = TimeFromExcel(j,dfs)

                    exitCostPut = trading_cost("Buy",exitPremPut)
                    totalCostPut = Entry_CostPut+exitCostPut
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                    
                    break

                if(TimeFromExcel(j,dfs) == stopTime):
                        exitPremPut = PutPremFromExcel2(iP,dfp)
                        
                        exitTimePut = TimeFromExcel(j,dfs)
                        

                        exitCostPut = trading_cost("Buy",exitPremPut)
                        totalCostPut = Entry_CostPut+exitCostPut

                        Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                        Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                        print("FINAL TABLE".center(200,'*'))
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


                        print(prev_table)
                        print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                        print(bufferZone)
                        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                
                        print("Day Finished")
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print("ALL DAYS FINISHED")
                                sys.exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            stopTime = day[0]+" 15:25:00"
                            startTime = day[0]+" 09:20:00"

                            j = findJ(startTime,dfs)
                            
                            start(j,"",0,count,dfc,dfp,dfs)
                        else:
                            print("ALL DAYS FINISHED")
                            sys.exit()
                            

                    
                    
            else:
                cmp = CMPFromExcel(j,dfs)
                if(PutPremFromExcel2(iP,dfp)*(rate+1)<stlp_1P):
                    stlp_1P=PutPremFromExcel2(iP,dfp)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremPut = PutPremFromExcel2(iP,dfp)
                    exitTimePut = TimeFromExcel(j,dfs)

                    exitCostPut = trading_cost("Buy",exitPremPut)
                    totalCostPut = Entry_CostPut+exitCostPut

                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
                    print("FINAL TABLE".center(200,'*'))
                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                    print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


                    print(prev_table)
                    print("|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                    print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                    print(bufferZone)
                    print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    print("Day Finished")
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print("ALL DAYS FINISHED")
                            sys.exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+" 09:20:00"

                        j = findJ(startTime,dfs)
                        

                        start(j,"",0,count,dfc,dfp,dfs)                    
                    else:
                        print("ALL DAYS FINISHED")
                        sys.exit()

                
    
                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)



            j=j+n
            iP =iP+n
            iC = iC+n

    session_total = (Real_Call_P_or_L +Real_Put_P_or_L)*qty

    x = "Total = "+str(total)


    prev_table = prev_table +"\n"+str(bufferZone)+"|"+PE.center(16)+"|"+str(exitTimePut).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|\n"+"|"+CE.center(16)+"|"+str(exitTimeCall).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|\n"
    total  = total+session_total

    if(TimeFromExcel(j,dfs) != str(stopTime)):
        count-=1
        
        if(j==initial):
            start(j+1,prev_table,total,count,dfc,dfp,dfs)
        else:
            start(j+1,prev_table,total,count,dfc,dfp,dfs)


    if(TimeFromExcel(j,dfs) == stopTime):
        
        print("FINAL TABLE".center(200,'*'))
        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |")


        print(prev_table)
        print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
        print(bufferZone)
        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")
        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                
        print("Day Finished")

        if(count<5 and j+37<dfs.iloc[-1,13] ):
            if(stopTime == final_stopTime):
                print("ALL DAYS FINISHED")
                sys.exit()
        
            t = TimeFromExcel(j+37,dfs)
            day = str(t).split()

            stopTime = day[0]+" 15:25:00"
            startTime = day[0]+" 09:20:00"

            j = findJ(startTime,dfs)


            
            start(j,"",0,count,dfc,dfp,dfs)
 
        else:
            print("ALL DAYS FINISHED")
            sys.exit()




In [ ]:
start(j,"",0,0,dfc,dfp,dfs)

In [ ]:
from threading import Thread
import threading

class CustomThread(Thread):
    def __init__(self, group=None, target=None, name=None,args={},kwargs ={},Verbose=None ):
        Thread.__init__(self, group,target,name,args,kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,**self._kwargs)

a=CustomThread(target=start,args=[j,"",0,0,dfc,dfp,dfs])
b=CustomThread(target=start,args=[j,"",0,0,dfc,dfp,dfs])
c=CustomThread(target=start,args=[j,"",0,0,dfc,dfp,dfs])
a.start()
b.start()
c.start()